## 最小ID

In [1]:
import numpy as np
from commons import how_long
from memory_profiler import profile

In [2]:
## 保证每次生成的一样
np.random.seed(1)
# 生成20万大小的id list
n=200000
ids = np.random.choice(range(n+2),n,replace=False)
if 0 not in ids:
    ids.append(0)
print(ids[1:4])

[171553   5506  38370]


>[171553   5506  38370]

In [3]:
## 暴力方法
@how_long
def min_id(ids):
    '''
    暴力方法
    ids:list of ids
    
    return min_id for create
    '''
    for i in range(len(ids)):
        if not_in(i,ids):
            return i
#     print(len(ids))
    return len(ids)

@how_long
def min_id_with_not_in(ids):
    for i in range(len(ids)):
        if i not in ids:
            return i
#     print(len(ids))
    return len(ids)
    

def not_in(i,ids):
    '''
    检查i是否在ids中，原本可以用
    if i not in ids
    为了保证使用最原始方法测试，所以自己用for重新写一个
    '''
    for one_id in ids:
        if i == one_id:
            return False
    return True

## 改进一

@how_long
def min_id_change_1(ids):
    '''
    改进一
    '''
    id_index = [False] * len(ids)
    for i in ids:
        if i < len(ids):
            id_index[i]=True
    for ii in range(len(id_index)):
        if id_index[ii] == False:
            return ii
#     print(len(ids))
    return len(ids)



## 改进二


def min_id_change_2_sub(l,u,ids):
    '''
    改进二
    '''
    if len(ids)==0:
        return l
    else:     
        middle = np.floor((l+u)/2)
        small_set=[]
        other_set=[]
        length = len(ids)
        for i in range(length):
            the_number,ids = ids[-1], ids[:-1]
            if the_number <= middle:
                small_set.append(the_number)
            else:
                other_set.append(the_number)
             
    if len(small_set) == middle+1-l:
        return min_id_change_2_sub(middle+1,middle+len(other_set),other_set)
    else:
        return min_id_change_2_sub(l,middle,small_set)
    return result
        


@how_long
def min_id_change_2(ids):
    '''
    递归方法
    '''
    result = min_id_change_2_sub(0,len(ids)-1,ids)
#     print(result)
    return result


## 改进二，不是用递归，而转化成迭代
@how_long
def min_id_change_2_iteration(ids):
    '''
    迭代方法
    '''
    n=len(ids)
    l=0
    u=n-1
    
    while n:
        left=0
        middle = np.floor((u+l)/2)
#         print(n,left,middle,l,u)
        for right in range(int(l),int(l+n)):
            if ids[right] <= middle:
                tmp = ids[right]
                ids[right]=ids[left+l]
                ids[left+l] =tmp
                left += 1
#             print(ids)
#         print('------')
#         print(ids)
#         print(left+l,middle - l + 1)
        if left == middle - l + 1:
#             print('left')
            l = int(middle + 1)
            u = u
            n = n-left
        else:
#             print('right')
            u = middle
            n = left
    return l



In [4]:
## 自己用for循环模仿 not in 
%memit print(min_id(ids))

min_id() cost:99173.9058 ms!
5192
peak memory: 69.01 MiB, increment: 0.06 MiB


In [5]:
## 使用 if i not in ids
%memit print(min_id_with_not_in(ids))

min_id_with_not_in() cost:371.5961 ms!
5192
peak memory: 69.02 MiB, increment: 0.00 MiB


In [6]:
## 改进一
%memit print(min_id_change_1(ids))

min_id_change_1() cost:68.5959 ms!
5192
min_id_change_1() cost:69.6430 ms!
5192
peak memory: 65.96 MiB, increment: 0.00 MiB


In [7]:
## 改进二 递归方法
%memit print(min_id_change_2(ids))

min_id_change_2() cost:36381.6068 ms!
5192.0
peak memory: 76.98 MiB, increment: 11.01 MiB


In [8]:
## 改进二 迭代方法
%memit print(min_id_change_2_iteration(ids))

min_id_change_2_iteration() cost:753.1919 ms!
5192
peak memory: 68.64 MiB, increment: 0.00 MiB


书中提到了，改进二，由于使用了递归，所以有一个调用的栈的大小消耗在内存中，所以会使得内存的消耗增大。

这还只是一个20万的id集合，而且查询的id只是5000左右，结果已经有很大差异了。

如果优化的好，迭代方法的确要比递归方法好。

<font color='red'>而且有一点，就是python中的内置方法`in`应该不是简单的for循环，或者是用c编译的，速度要快很多。所以在使用过程中，能使用已有的内置方法，最好就直接使用，应该会比自己写的要快。</font>

## 丑数

利用队列这种数据结构来进行丑数的查找。

**需要注意的是，不是技术刚好到达n就可以停下，而是要确保最小的的计数是n才可以**

不然会出现计数不对的情况，

```
[1,2,3,5]
其实这中间应该还有一个4，[1,2,3,4,5]，但是使用queue计算的时候大小关系并不是很有顺序
只能确定最小的一定是最小的，之后插入的一定不会比最小的小（因为是最小的*2,*3,*5得到的）
```

In [9]:
## 暴力方法

@how_long
def ugly_number_in_for(n):
    '''
    使用for暴力筛选丑数
    '''
    number = 0
    count = 0
    while count < n:
        number += 1
#         print(number,count)
        if is_ugly(number):
#             print(number)
            count +=1            
    return number
  
def is_ugly(n):
    '''
    判断是否为丑数
    '''
    while n % 2 ==0:
        n = n/2
    while n % 3 ==0:
        n = n/3
    while n% 5 ==0:
        n = n/5
    if n== 1:
        return True
    else:
        return False

@how_long
def ugly_number_queue(n):
    '''
    使用queue
    '''
    count =1
    queue=[1]
    has_dis =[]
    while count<n:
        number = queue.pop(0)
        fn = len(queue)
        for mult in [2,3,5]:
            tmp = mult * number
            if tmp not in has_dis:
                has_dis.append(tmp)
                queue.append(tmp)
        queue=sorted(set(queue))
#         print(queue)
#         print(queue,count)
        an = len(queue)
        count += 1
    return queue[0]


@how_long
def ugly_number_dif_queue(n):
    '''
    分队列queue
    '''
    count = 1
    min_n=1
    queue2=[2]
    queue3=[3]
    queue5=[5]
    while count<n:
        min_n = min(queue2[0],queue3[0],queue5[0])
        if queue2[0] == min_n:
            queue2.pop(0)
            queue2.append(2*min_n)
            queue3.append(3*min_n)
            queue5.append(5*min_n)
        elif queue3[0] == min_n:
            queue3.pop(0)
            queue3.append(3*min_n)
            queue5.append(5*min_n)
        else:
            queue5.pop(0)
            queue5.append(5*min_n)
        count += 1
        
    sort_queue=sorted(queue2+queue3+queue5)
    return min_n

In [10]:
print(ugly_number_in_for(500))

ugly_number_in_for() cost:560.6701 ms!
937500


In [11]:
print(ugly_number_queue(500))

ugly_number_queue() cost:23.5658 ms!
937500


In [12]:
print(ugly_number_dif_queue(500))

ugly_number_dif_queue() cost:0.4091 ms!
937500


**差异很明显啊**